In [31]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as ns
import xml.etree.ElementTree as ET
import time

## Reding xml data into pandas dataframe

In [26]:
def read_xml(xml_filename):
    xtree = ET.parse(xml_filename)
    xroot = xtree.getroot() 
    
    # getting the columns
    df_cols = []
    for x in xroot:
        for attr in x.attrib:
            if attr not in df_cols:
                df_cols.append(attr)
    
    # getting the records
    rows = []
    for x in xroot:
        row = dict()
        for col in df_cols:
            if col in x.attrib:
                row[col] = x.attrib[col]
            else:
                row[col] = None
        rows.append(row)
        
    out_df = pd.DataFrame(rows, columns = df_cols)
    return out_df

In [29]:
%%time
badges = read_xml("data/Badges.xml")
print("badges:", badges.shape)
comments = read_xml("data/Comments.xml")
print("comments:", comments.shape)
post_history = read_xml("data/PostHistory.xml")
print("post_history:", post_history.shape)
post_links = read_xml("data/PostLinks.xml")
print("post_links:", post_links.shape)
posts = read_xml("data/Posts.xml")
print("posts:", posts.shape)
tags = read_xml("data/Tags.xml")
print("tags:", tags.shape)
users = read_xml("data/Users.xml")
print("users:", users.shape)
votes = read_xml("data/Votes.xml")
print("votes:", votes.shape)

badges: (78180, 6)
comments: (41722, 7)
post_history: (121522, 9)
post_links: (1653, 5)
posts: (39929, 21)
tags: (489, 5)
users: (66951, 13)
votes: (125650, 6)
Wall time: 6.78 s


In [32]:
start_time = time.time()

In [34]:
print("It took %s seconds to process" % (time.time() - start_time))

It took 24.416924238204956 seconds to process
